<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Agentic-RAG-Streamlit-App/blob/main/agent_rag_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# agentic_rag_pipeline.py
import os
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
import logging

# -----------------------------
# 1️⃣ Загрузка переменных окружения
# -----------------------------
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "./chroma_db")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("AgenticRAG")

# -----------------------------
# 2️⃣ Настройка векторного хранилища
# -----------------------------
def create_vectordb(documents):
    """Создает векторное хранилище из документов"""
    embeddings = OpenAIEmbeddings(openai_api_key=GROQ_API_KEY)
    vectordb = Chroma.from_documents(documents, embeddings, persist_directory=CHROMA_DB_PATH)
    vectordb.persist()
    return vectordb

# -----------------------------
# 3️⃣ Определяем инструменты агента
# -----------------------------
class TavilySearchTool(BaseTool):
    """Простой интерфейс для web search через Tavily"""
    name = "TavilySearch"
    description = "Используется для поиска информации в интернете"

    def _call(self, query: str) -> str:
        # Здесь можно интегрировать реальный Tavily API
        logger.info(f"Tavily search query: {query}")
        # Заглушка для демонстрации
        return f"Результат поиска Tavily для запроса: {query}"

# -----------------------------
# 4️⃣ Создаем агент с инструментами
# -----------------------------
def create_agent(vectordb):
    """Создает агента, который может использовать инструменты и RAG"""
    # LLM
    llm = ChatOpenAI(openai_api_key=GROQ_API_KEY, temperature=0)

    # Retrieval QA
    retriever = vectordb.as_retriever()
    retrieval_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    # Инструменты агента
    tools = [
        Tool(name="RAG", func=retrieval_chain.run, description="Поиск по базе знаний"),
        TavilySearchTool()
    ]

    # Инициализация агента
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
    return agent

# -----------------------------
# 5️⃣ Python API для Streamlit
# -----------------------------
class AgenticRAG:
    def __init__(self, documents):
        self.vectordb = create_vectordb(documents)
        self.agent = create_agent(self.vectordb)

    def get_answer(self, query: str) -> str:
        try:
            logger.info(f"Запрос пользователя: {query}")
            answer = self.agent.run(query)
            logger.info(f"Ответ агента: {answer}")
            return answer
        except Exception as e:
            logger.error(f"Ошибка при выполнении агента: {e}")
            return f"Произошла ошибка: {str(e)}"
